### Time Series

### TS_data.csv

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings("ignore")
np.random.seed(42)
%matplotlib inline

In [2]:
from sklearn import datasets
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics
from pandas.api.types import is_string_dtype, is_numeric_dtype
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split,cross_val_score,StratifiedKFold

In [3]:
from fbprophet import Prophet

ModuleNotFoundError: No module named 'fbprophet'

In [4]:
#!pip install pystan
#!pip install fbprophet
#!pip install fbprophet

^C


#### Read the csv file

In [49]:
dataset=pd.read_csv("C:\\Users\\Rebecca\\Documents\\TL Study Docs\\ML\\Dataset\\Automobile_data.csv",low_memory=False)

In [50]:
dataset.shape

(205, 26)

In [51]:
dataset.head()

,symboling,normalized-losses,make,fuel-type,aspiration,num-of-doors,body-style,drive-wheels,engine-location,wheel-base,...,engine-size,fuel-system,bore,stroke,compression-ratio,horsepower,peak-rpm,city-mpg,highway-mpg,price
0,3,?,alfa-romero,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495
1,3,?,alfa-romero,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500
2,1,?,alfa-romero,gas,std,two,hatchback,rwd,front,94.5,...,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500
3,2,164,audi,gas,std,four,sedan,fwd,front,99.8,...,109,mpfi,3.19,3.4,10.0,102,5500,24,30,13950
4,2,164,audi,gas,std,four,sedan,4wd,front,99.4,...,136,mpfi,3.19,3.4,8.0,115,5500,18,22,17450


In [52]:
dataset=dataset.replace('?',np.nan)

###### Extract all string columns

In [53]:
col_string=[]
for col in dataset:
    if is_string_dtype(dataset[col]):
        col_string.append(col)
print("String type columns: ",col_string)

String type columns:  ['normalized-losses', 'make', 'fuel-type', 'aspiration', 'num-of-doors', 'body-style', 'drive-wheels', 'engine-location', 'engine-type', 'num-of-cylinders', 'fuel-system', 'bore', 'stroke', 'horsepower', 'peak-rpm', 'price']


convert following to continuos data

In [54]:
cols=['normalized-losses','bore', 'stroke', 'horsepower', 'peak-rpm', 'price']

for i in cols:
    dataset[i]=pd.to_numeric(dataset[i],errors='raise')

convert following to categorical data

In [55]:
for i in dataset:
    if is_string_dtype(dataset[i]):
        dataset[i]=dataset[i].astype('category').cat.as_ordered()

In [56]:
dataset[:3]

,symboling,normalized-losses,make,fuel-type,aspiration,num-of-doors,body-style,drive-wheels,engine-location,wheel-base,...,engine-size,fuel-system,bore,stroke,compression-ratio,horsepower,peak-rpm,city-mpg,highway-mpg,price
0,3,NaN,alfa-romero,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111.0,5000.0,21,27,13495.0
1,3,NaN,alfa-romero,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111.0,5000.0,21,27,16500.0
2,1,NaN,alfa-romero,gas,std,two,hatchback,rwd,front,94.5,...,152,mpfi,2.68,3.47,9.0,154.0,5000.0,19,26,16500.0


In [57]:
for i in dataset:
    if is_numeric_dtype(dataset[i]):
        col_vals=dataset[i]
        if(sum(col_vals.isnull()))!=0:
            dataset[i+'_na']=col_vals.isnull()
            dataset[i]=col_vals.fillna(col_vals.median())       
        

In [58]:
dataset[:1]

,symboling,normalized-losses,make,fuel-type,aspiration,num-of-doors,body-style,drive-wheels,engine-location,wheel-base,...,peak-rpm,city-mpg,highway-mpg,price,normalized-losses_na,bore_na,stroke_na,horsepower_na,peak-rpm_na,price_na
0,3,115.0,alfa-romero,gas,std,two,convertible,rwd,front,88.6,...,5000.0,21,27,13495.0,True,False,False,False,False,False


In [59]:
for i in dataset:
    if str(dataset[i].dtype)=='category':
        print(dataset[i].dtype)
        dataset[i]=dataset[i].cat.codes+1

category
category
category
category
category
category
category
category
category
category


In [60]:
X=dataset.drop('price',axis=1)
Y=dataset['price']
Xtrain,Xtest,Ytrain,Ytest=train_test_split(X,Y,test_size=0.2,random_state=99)

In [61]:
m=RandomForestRegressor(n_jobs=-1)
m.fit(Xtrain,Ytrain)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=-1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [65]:
import math
def rmse (preds,actuals):
    return math.sqrt(((preds-actuals)**2).mean())

In [66]:
[rmse(m.predict(Xtrain),Ytrain),rmse(m.predict(Xtest),Ytest),m.score(Xtrain,Ytrain),m.score(Xtest,Ytest)]

[1441.9943898999913,
 2677.3417821782787,
 0.9679678167083768,
 0.8466183272263432]

#### Predict Categorical -Symboling

In [67]:
X=dataset.drop('symboling',axis=1)
Y=dataset['symboling']
Xtrain,Xtest,Ytrain,Ytest=train_test_split(X,Y,test_size=0.2,random_state=99)

In [68]:
m=RandomForestClassifier(n_jobs=-1)
m.fit(Xtrain,Ytrain)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=-1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [69]:
print(m.score(Xtrain,Ytrain))
print(m.score(Xtest,Ytest))

1.0
0.8048780487804879


###### Parameter Tuning

In [70]:
from sklearn.model_selection import RandomizedSearchCV
from pprint import pprint

In [71]:
n_estimators=[int(x) for x in np.linspace(start=20,stop=200,num=9)]
max_features=['auto','sqrt']
max_depth=[int(x) for x in np.linspace(1,45,num=3)]
min_samples_split=[5,10]
random_grid={'n_estimators':n_estimators,'max_features':max_features,
            'max_depth':max_depth,
            'min_samples_split':min_samples_split}

pprint(random_grid)

{'max_depth': [1, 23, 45],
 'max_features': ['auto', 'sqrt'],
 'min_samples_split': [5, 10],
 'n_estimators': [20, 42, 65, 87, 110, 132, 155, 177, 200]}


In [72]:
forest=RandomForestClassifier(n_jobs=-1)

In [74]:
rf_random=RandomizedSearchCV(estimator=forest,param_distributions=random_grid,n_iter=10,cv=10,random_state=99,n_jobs=-1,scoring='neg_mean_squared_error')
rf_random.fit(Xtrain,Ytrain)

RandomizedSearchCV(cv=10, error_score='raise-deprecating',
          estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=-1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
          fit_params=None, iid='warn', n_iter=10, n_jobs=-1,
          param_distributions={'n_estimators': [20, 42, 65, 87, 110, 132, 155, 177, 200], 'max_features': ['auto', 'sqrt'], 'max_depth': [1, 23, 45], 'min_samples_split': [5, 10]},
          pre_dispatch='2*n_jobs', random_state=99, refit=True,
          return_train_score='warn', scoring='neg_mean_squared_error',
          verbose=0)

In [75]:
cvress2=rf_random.cv_results_

In [76]:
rf_random.best_estimator_

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=45, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=10,
            min_weight_fraction_leaf=0.0, n_estimators=65, n_jobs=-1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [78]:
rf_random.best_params_

{'n_estimators': 65,
 'min_samples_split': 10,
 'max_features': 'auto',
 'max_depth': 45}

In [80]:
from sklearn.metrics import mean_squared_error
random_best=rf_random.best_estimator_.predict(Xtrain)
final_mse=mean_squared_error(Ytrain,random_best)
final_rmse=np.sqrt(final_mse)
print('The best model from randomized search has a RMSE of',round(final_rmse,2))

The best model from randomized search has a RMSE of 0.36


#### Auto Tuning Sample

In [82]:
numofestimators=[1,5,10,20,25,30]
numofleafs=[1,3,5,10,25]
numoffeatures=np.arange(0.1,1.1,0.1)

best_result=[]

for Numofestimator in numofestimators:
    for Numofleafs in numofleafs:
        for numoffeatures in numoffeatures:
            result=[Numofestimator,Numofleafs,numoffeatures]
            m=RandomForestClassifier(n_jobs=-1,n_estimators=Numofestimator,min_samples_leaf=Numofleafs,
                                    max_features=numoffeatures)
            m.fit(Xtrain,Ytrain)
            result.append(m.score(Xtrain,Ytrain))
            result.append(m.score(Xtest,Ytest))
            if len(best_result)==0:
                best_result=result
            elif best_result[4]<result[4]:
                print(result)
                best_result=result

print('Best Result',best_result)

[1, 1, 0.2, 0.9085365853658537, 0.7073170731707317]
[1, 1, 0.6, 0.8841463414634146, 0.7317073170731707]
[1, 1, 0.7000000000000001, 0.9390243902439024, 0.8292682926829268]


TypeError: 'numpy.float64' object is not iterable

In [ ]:
from sklearn.model_selection import GridSearchCV
param_grid=[{
    'n_estimators':[1,5,10,15,20,25,30],'max_features':[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.],
    'min_samples_leaf':[1,3,5,10,25],'bootstarp':[True,False]
}]

forest=RandomForestClassifier(n_jobs=-1)
grid_search_forest=GridSearchCV(forest,param_grid,cv=10,scoring='neg_mean_sqaured_error')
grid_search_forest.fit(Xtrain,Ytrain)